In [1]:
import pandas as pd
import time

In [2]:
df_train = pd.read_csv("data/technical/sales_train.csv", parse_dates=['date'])

In [3]:
all_date_stores_open = df_train[['date']].copy().drop_duplicates()

In [4]:
all_date_stores_open.describe()

/var/folders/qp/tyy4nbzs0l71vykwhq03d71r0000gn/T/ipykernel_30033/810798551.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  all_date_stores_open.describe()


,date
count,1034
unique,1034
top,2013-02-01 00:00:00
freq,1
first,2013-01-01 00:00:00
last,2015-12-10 00:00:00


In [5]:
all_date_stores_open.head()

,date
0,2013-02-01
1,2013-03-01
2,2013-05-01
3,2013-06-01
4,2013-01-15


In [6]:
all_ids_for_submission = pd.read_csv("data/technical/sample_submission.csv")

In [7]:
all_ids_for_submission = all_ids_for_submission['ID']

In [8]:
timestamp = time.time()

matrix = []

# create a matrix containing all the time series for each and every item sold that
# needs to be predicted.
# start off by finding all records of sold items and on which day.
# then merge on a list containing all the dates that have to be taken into account
# during the period of the time series
# then reset and sort the index.
# finally, add the tuple of the item_id and the dataframe containing all records into
# the final matrix
for item in all_ids_for_submission:
    item_transactions = df_train.loc[df_train['item_id'] == item]
    item_transactions = item_transactions[['date', 'item_cnt_day']].copy()
    temp_date_df = all_date_stores_open
    temp_df = pd.merge(item_transactions, temp_date_df, how = 'outer', on=['date'])
    temp_df.fillna(0, inplace=True)
    temp_df.set_index('date', inplace=True, drop=True)
    temp_df.sort_index(inplace=True)
    matrix.append((item, temp_df))

print(time.time() - timestamp)

1228.264605998993
